In [ ]:
import spacy

flter_for_kb_entities=False #True if you want to use only entities existing in KB from MedMentions
filter_for_entity_in_annot=False #True if yu only want to use found entities that are annotated in MedMentions

use_own_biobert=False #True if you want to use Biobert as NER
use_own_NER=False     #True if you want to use own NER based on MedMentions 
#if both are false, the whole pretrained scispacy pipeline from medmentions_and_training.ipynb will be used

#model paths
#ownBio:________________biobert/named-entity-recognition/output/all/checkpoint-XXXXX/
#ownNER:________________NERdemo/output/model-best
#wandBSweepNER:_________wandBSweep/output/model-best

#path to trained EL:____model/name_of_trained_model with medmentions_and_training.ipynb

path_to_NER_model="./NERdemo/output/model-best"

path_to_NEL_model="./models/my_nlp_FinalKB"

load medmentions

In [ ]:
#need a existing kb to filter for entities. specify path to kb
#increases runtime of next cell 6 dramatically

if flter_for_kb_entities:
    import spacy
    from spacy.kb import InMemoryLookupKB
    nlp = spacy.load("en_core_sci_lg")
    kb = InMemoryLookupKB(vocab=nlp.vocab, entity_vector_length=200)
    kb.from_disk(knowledgebase_address)
    enti=kb.get_entity_strings()

In [ ]:
def read_lines(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip()

In [ ]:
filter_cuis=False
dataset = []
file_path='./MedMentions-master/MedMentions-master/full/data/corpus_pubtator.txt'
same_pmid=False
abstract=False
for line in read_lines(file_path):
    if len(line.split("|")) == 3 and same_pmid == False:
        pmid, type, text = line.split("|")
        if type == 't':
            same_pmid=True
            abstract = False
            mentions={}
            entities=[]
            cuis=[]
    if len(line.split("|")) == 3 and same_pmid == True:
        abstract = line.split("|")[2]
        text_and_abstract=text + ' ' + abstract
        abstract=True
    if len(line.split('\t')) == 6 and same_pmid == True :
        pmid, start, end, mention, semId, cui = line.split('\t')
        if flter_for_kb_entities:
            if cui not in enti:
                continue
        offset=(int(start),int(end))
        links_dict={cui:1.0}
        mentions[offset]=links_dict
        entities.append((int(start),int(end),cui))
        cuis.append(cui)
        abstract=False
        
    if len(line.split('\t')) == 1 and same_pmid == True and not abstract:
        same_pmid = False
        dataset.append((text_and_abstract,{"links":mentions,"entities":entities},pmid ))


In [ ]:
train_dataset = []
test_dataset = []
pmids_train=[]
file_path_train='./MedMentions-master/MedMentions-master/full/data/corpus_pubtator_pmids_trng.txt'

for line in read_lines(file_path_train):
    pmids_train.append(line)

pmids_test=[]
file_path_test='./MedMentions-master/MedMentions-master/full/data/corpus_pubtator_pmids_test.txt'
for line in read_lines(file_path_test):
    pmids_test.append(line)

for pmid_train in pmids_train:
    train_dataset.extend(data for data in dataset if data[2]==pmid_train)

for pmid_test in pmids_test:
    test_dataset.extend(data for data in dataset if data[2]==pmid_test)

In [ ]:
if use_own_biobert:
    from spacy.tokens import Doc
    from spacy.language import Language
    from spacy.tokens import Span

    from safetensors.torch import load_model
    from transformers import AutoTokenizer, AutoModelForTokenClassification
    from transformers import pipeline


    tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1", model_max_length=512)
    model = AutoModelForTokenClassification.from_pretrained(path_to_NER_model)

    ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first")
    # Function to integrate BioBERT model into spaCy pipeline
    @Language.component('bio')
    def ner_bioBERT(doc):
        entities=ner(doc.text)
        for ent in entities:
            span=doc.char_span(ent["start"], ent["end"], label=ent["entity_group"])
            try:
                doc.set_ents(entities=[span],default="unmodified")
            except:
                continue

        #doc.ents = [Span(doc, start, end, label) for start, end, label in entities]
        return doc
    nlp_nel = spacy.load(path_to_NEL_model)
    nlp = spacy.blank("en")
    nlp.add_pipe("bio", last=True)
    nlp.add_pipe("sentencizer", name="sent", source=nlp_nel)
    nlp.add_pipe("entity_linker", name="nel", source=nlp_nel)
elif use_own_NER:
    nlp_nel = spacy.load(path_to_NEL_model)
    nlp = spacy.load(path_to_NER_model)
    nlp.add_pipe("sentencizer", name="sent", source=nlp_nel)
    nlp.add_pipe("entity_linker", name="nel", source=nlp_nel)
else:
    nlp = spacy.load(path_to_NEL_model)

In [ ]:
detected=[]
annot=[]
for text, true_annot, _ in test_dataset:
  doc = nlp_ner(text)
  entitiesListGOLD=set()
  for link in true_annot["links"].items():
    start=link[0][0]
    end=link[0][1]
    key, _ = list(link[1].items())[0]
    entitiesListGOLD.add((start,end,key))
  entitiesList=set()
  for ent in doc.ents:
    entitiesList.add((ent.start_char,ent.end_char,ent.kb_id_))
  detected.append(entitiesList)
  annot.append(entitiesListGOLD)

if filter_for_entity_in_annot:
  cleaned=[]
  for i, detect in enumerate(detected):
    entitiesListGOLD=set()
    for val in detect:
      for entr in annot[i]:
        if val[0] == entr[0] and val[1] ==entr[1]:
          entitiesListGOLD.add(val)
        else:
          continue
    cleaned.append(entitiesListGOLD)
  

tp=0
fn=0
fp=0

if filter_for_entity_in_annot:
  for i, enti in enumerate(cleaned):
    tp=tp+len(enti.intersection(annot[i]))
    fp=fp+len(enti - annot[i])
    fn=fn+len(annot[i] - enti)
else:
  for i, enti in enumerate(detected):
    tp=tp+len(enti.intersection(annot[i]))
    fp=fp+len(enti - annot[i])
    fn=fn+len(annot[i] - enti)

recall=tp/(tp+fn)
precision=tp/(tp+fp)
f1=2*((precision*recall)/(precision+recall))
print(recall)
print(precision)
print(f1)

In [ ]:
detected=[]
annot=[]
for text, true_annot, _ in test_dataset:
   # print(true_annot["links"][(0,20)])
    doc = nlp_ner(text)  # to make this more efficient, you can use nlp.pipe() just once for all the texts
    entitiesListGOLD=set()
    for link in true_annot["links"].items():
      key, _ = list(link[1].items())[0]
      entitiesListGOLD.add(key)
    entitiesList=set()
    for ent in doc.ents:
       entitiesList.add(ent.kb_id_)
    detected.append(entitiesList)
    annot.append(entitiesListGOLD)

    
if filter_for_entity_in_annot:
  cleaned=[]
  for i, detect in enumerate(detected):
    entitiesListGOLD=set()
    for val in detect:
      for entr in annot[i]:
        if val[0] == entr[0] and val[1] ==entr[1]:
          entitiesListGOLD.add(val)
        else:
          continue
    cleaned.append(entitiesListGOLD)    
    
tp=0
fn=0
fp=0

if filter_for_entity_in_annot:
  for i, enti in enumerate(cleaned):
    tp=tp+len(enti.intersection(annot[i]))
    fp=fp+len(enti - annot[i])
    fn=fn+len(annot[i] - enti)
else:
  for i, enti in enumerate(detected):
    tp=tp+len(enti.intersection(annot[i]))
    fp=fp+len(enti - annot[i])
    fn=fn+len(annot[i] - enti)

recall=tp/(tp+fn)
precision=tp/(tp+fp)
f1=2*((precision*recall)/(precision+recall))
print(recall)
print(precision)
print(f1)
